### Solving Psychopath Mazes!

There's a really interesting maze game that was published in 2005.  It's called Psychopath.

Check it out here!:
http://www.k2xl.com/games/psychopath/index2.php

#Unfortunately, as of writing, it is impossible to register for this version, so if you don't have an account, it's going to be really aggravating trying to keep your progress saved.  That said, if you already have an account, this is the coolest version to play the game on and you should stick with it.

#However, if not, you can register for psychopath2... and you'll have access to all of the Psychopath 1 levels and can see any levels that I am talking about...

http://www.k2xl.com/games/psychopath2/


As of writing, only 39 users have cleared all of the levels in the original game.  This is partly due to the fact that it is impossible to register as of a few years ago.  

I am one of the people who has cleared all the levels, and while it's interesting to solve a bunch of mazes using logic and intuition as a person, I'm excited to attempt what I believe to be far more difficult -- writing a bot that could also beat all of the levels.

The first few levels aren't all that interesting and are just like a regular maze that a mouse might
learn to solve in its search for cheese or peanut butter, but in the double digits, ESPECIALLY around level 30,
things get extremely interesting due to the addition of movable blocks.

Below is the code for the solver itself.  Scrolling down all the way past that, I will be covering how it solves certain types of levels.  To begin reading that part, search for ###Beginning

In [2]:
import numpy as np
from copy import copy

In [4]:
class Maze:
    
    def __init__(self,level=1):
        self.level = level
        self.win = False
        self.exits,self.player_position,self.obstructions,self.movable_blocks,\
        self.width,self.height,self.solution_moves,self.maze =\
        self.load_maze_from_file(level)
        
    def print_maze(self,small=True):
        if small:
            for x in self.maze:
                print("".join(x))
        else:
            for x in self.maze:
                print(x)
    
    def load_maze_from_file(self,level=1):
        with open("maps/{}.txt".format(level)) as file:
            lines = [x.strip() for x in file.readlines()]
        #print(lines) 
        width,height = [int(x) for x in lines[0].split()]
        #print(width,height) 
        solution_moves = int(lines[1])
        #print(solution_moves)
        board = [] 
        idx = 2
        for _ in range(height):
            board.append(list(lines[idx]))
            idx += 1
            
        exits = []
        obstructions = False
        movable_blocks = False
        for row in range(height):
            for col in range(width):
                if board[row][col] == "u":
                    player_position = [row,col]
                elif board[row][col] == "e":
                    exits.append([row,col])
                elif board[row][col] == "p":
                    obstructions = True
                    movable_blocks = True
                elif board[row][col] == "x":
                    obstructions = True

        return exits,np.array(player_position),obstructions,movable_blocks,width,height,solution_moves,board

    def move_player(self,direction):
        '''
        returns False if could not move that way
        returns True if moved that way.  Maze will be updated afterwards.
        '''
        
        if direction == "L":
            updates = [0,-1]
        elif direction == "R":
            updates = [0,1]
        elif direction == "U":
            updates = [-1,0]
        elif direction == "D":
            updates = [1,0]
        
        destination = self.player_position + updates
        after_dest = destination + updates
        #print(destination)
        #print(after_dest)
        obj_at_dest = self.maze[destination[0]][destination[1]]
        obj_after_dest = self.maze[after_dest[0]][after_dest[1]]
        #print(obj_at_dest)
        #print(obj_after_dest)
        
        def obj_swap_code(pos1,pos2,player_pos1=True,win=False,covering_exit=False):
            if player_pos1:
                self.player_position = pos2
            if win:
                self.maze[pos1[0]][pos1[1]] = "_"
                self.maze[pos2[0]][pos2[1]] = "W"
            elif covering_exit:
                #should be entered as destination,after_dest
                self.maze[pos1[0]][pos1[1]] = "_"
                self.maze[pos2[0]][pos2[1]] = "P"
            else:
                self.maze[pos1[0]][pos1[1]],self.maze[pos2[0]][pos2[1]] =\
                self.maze[pos2[0]][pos2[1]],self.maze[pos1[0]][pos1[1]] 
                
        #self.print_maze() 
        
        #player wants to move into empty space
        if obj_at_dest == "_":
            obj_swap_code(self.player_position,destination)
            return True
        
        #player attempts to move into a wall
        elif obj_at_dest in ["x","X"]:
            return False
        
        #player wants to move to visible exit and win the game
        elif obj_at_dest == "e":
            self.win = True
            obj_swap_code(self.player_position,destination,win=True)
            return True
        
        #player wants to move to where a regular movable block is
        elif obj_at_dest == "p":
            if obj_after_dest == "_":
                obj_swap_code(destination,after_dest,player_pos1=False)
                obj_swap_code(self.player_position,destination)
                return True
            elif obj_after_dest == "e":
                obj_swap_code(destination,after_dest,player_pos1=False,covering_exit=True)
                obj_swap_code(self.player_position,destination)
                return True
            elif obj_after_dest in ["p","P","x","X"]:
                return False
            
        #player wants to move to where a movable block that is covering an exit is
        elif obj_at_dest == "P":
            if obj_after_dest in ["_","e"]:
                self.win = True
                obj_swap_code(self.player_position,destination,win=True)
                return True
            elif obj_after_dest in ["p","P","x","X"]:
                return False
            
    def get_maze_without_movable_blocks(self):
        mz = Maze(self.level)
        for row in range(mz.height):
            for col in range(mz.width):
                if mz.maze[row][col] == "p":
                    mz.maze[row][col] = "_"
        return mz
            
    def check_win_from_list(self,move_list):
        '''takes a list of moves and tells you if you have found a solution to the maze
        example list ["L","L","U","L","L","L","D","R"]
        (left left up left left left down right)'''
        mz = Maze(self.level)
        if len(move_list) != mz.solution_moves:
            return False
        else:
            moves_made = 0
            max_moves = mz.solution_moves
            while moves_made != mz.solution_moves:
                move_made = mz.move_player(move_list[moves_made])
                if not move_made:
                    break
                moves_made += 1
            return mz.win
                

In [5]:
class Pathfinder:
    
    def __init__(self):
        pass
    
    def solve_maze(self,maze):
        if not maze.obstructions:
            return self.noobs_path(maze)
        elif not maze.movable_blocks:
            return self.bfs_path(maze)
        else: #hard one with movable blocks
            pass
    
    def noobs_path(self,maze):
        start = maze.player_position #[2,7]
        end = maze.exits[0]

        movement = end-start
        #print(movement,movement[0],movement[1])
        output = []
        if movement[0] > 0:
            output += ["D"]*movement[0]
        elif movement[0] < 0:
            output += ["U"]*abs(movement[0])
        #print(output)
        if movement[1] > 0:
            output += ["R"]*movement[1]
        elif movement[1] < 0:
            output += ["L"]*abs(movement[1])
        #print(output)

        return output
    
    def bfs_path(self,mz):

        ################# creating graph structure
        connections = {}

        reachable = ["_","u","e"]

        for row in range(mz.height):
            for col in range(mz.width):
                if mz.maze[row][col] in reachable:
                    loc = np.array([row,col])
                    connected_to = []
                    updates = [[0,-1],[0,1],[-1,0],[1,0]]
                    for update in updates:
                        conn = loc + update
                        if mz.maze[conn[0]][conn[1]] in reachable:
                            connected_to.append(tuple(conn))
                    connections[tuple(loc)] = connected_to

        #print(connections[tuple(mz.player_position)]) 
        #print(connections[(3, 8)])  


        ############### bfs'ing to get nodes at all distances from start
        start = tuple(mz.player_position) 
        end = tuple(mz.exits[0]) 
        #print(start) 
        #print(end) 


        visited = {start}
        to_explore = set(connections[start])
        #print(visited)
        #print(to_explore) 

        layers = {}
        layers[0] = {start}
        layers[1] = set(connections[start])

        layer_counter = 2
        while True:
            nx_layer = set()
            while True:
                nx = to_explore.pop()
                for conn in connections[nx]:
                    if conn not in visited:
                        visited.add(conn)
                        nx_layer.add(conn)
                layers[layer_counter] = nx_layer
                if len(to_explore) == 0:
                    to_explore = copy(nx_layer)
                    layer_counter += 1
                    break
            if end in to_explore:
                break

        #From this layer until reached layers[0], we can look for nodes that the exit is connected to all the way up,
        # and it should find a path upward.
        #print(layers[mz.solution_moves-1]) 

        #print(layers) 

        ############backtracking from end to get path

        layer_counter = mz.solution_moves
        current_position = end

        output = []
        layer_counter -= 1
        #print("init_LC",layer_counter)
        while layer_counter != -1:
            nx_layer = layers[layer_counter]
            for conn in connections[current_position]:
                if conn in nx_layer:
                    nx_position = conn
                    break
            #print(current_position,layer_counter,layers[layer_counter])
            movement = tuple(np.array(nx_position)-current_position)
            mov_dict = {(0,1):"L",(0,-1):"R",(1,0):"U",(-1,0):"D"}
            output.append(mov_dict[movement])
            current_position = nx_position
            layer_counter -= 1

        output.reverse()
        return output
    
    def cheese_bfs(self,mz):
        cheese_maze = mz.get_maze_without_movable_blocks()
        output = self.bfs_path(cheese_maze)
        return output

### ###Beginning -- Maps with no obstructions

In [6]:
Maze().maze

[['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X'],
 ['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X'],
 ['X', '_', '_', '_', '_', '_', '_', 'u', '_', 'X'],
 ['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X'],
 ['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X'],
 ['X', '_', 'e', '_', '_', '_', '_', '_', '_', 'X'],
 ['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X'],
 ['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']]

This is what level 1 of Psychopath looks like.  While it is impossible to exit the "maze" by trying to move past the edges, there is nothing that could potentially obstruct any shortest manhattan path from 'u' from the exit 'e'.

This means that a path is really simple to figure out.

In [7]:
mz = Maze() 
mz.print_maze(small=False)  
path = Pathfinder().solve_maze(mz) 
print()
print("An optimal solution to this maze",path) 

['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']
['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['X', '_', '_', '_', '_', '_', '_', 'u', '_', 'X']
['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['X', '_', 'e', '_', '_', '_', '_', '_', '_', 'X']
['X', '_', '_', '_', '_', '_', '_', '_', '_', 'X']
['X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X']

An optimal solution to this maze ['D', 'D', 'D', 'L', 'L', 'L', 'L', 'L']


### Traditional Mazes -- immovable blocks are in the way.

In [8]:
mz = Maze(4)
mz.print_maze() 

XXXXXXXXXXXXXXXXX
X_______________X
X________xxxx___X
X______x___ux___X
X_____x_____x___X
X____x___x__x___X
X___x__x__x_x___X
X____x__x___x___X
X_____x_____x___X
X___e____x______X
X__________x____X
X_______________X
XXXXXXXXXXXXXXXXX


In order to solve these, we can represent the maze as a graph and implement Breadth First Search (BFS).  We're using this, and not something like Dijkstra's Algorithm, or the well-known improvement on Dijkstra called A*, because it's completely unnecessary.  Those algorithms exist because in many graphs, there are costs associated with different paths.  But if the costs are all the same, you can just BFS or flood fill or however you want to think of it.

Below, I'll be showing solutions for levels 2 through 9 inclusive, which only contain a player with a start location, one exit, and some immovable blocks.

In [9]:
def show_solution(level=1):
    mz = Maze(level)
    mz.print_maze() 
    path = Pathfinder().solve_maze(mz) 
    print()
    print("An optimal solution to this maze",path) 

In [10]:
show_solution(2) 

XXXXXXXXXXXXXXX
X_____________X
X_____________X
X__u___x___e__X
X_____________X
X_____________X
XXXXXXXXXXXXXXX

An optimal solution to this maze ['U', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'R', 'R']


In [11]:
show_solution(3) 

XXXXXXXXXXX
X_________X
X_u_______X
X____x____X
X_______x_X
X___xx_x__X
X____x____X
X______xe_X
X_________X
XXXXXXXXXXX

An optimal solution to this maze ['D', 'D', 'R', 'R', 'R', 'R', 'D', 'D', 'R', 'R', 'D']


In [12]:
show_solution(4) 

XXXXXXXXXXXXXXXXX
X_______________X
X________xxxx___X
X______x___ux___X
X_____x_____x___X
X____x___x__x___X
X___x__x__x_x___X
X____x__x___x___X
X_____x_____x___X
X___e____x______X
X__________x____X
X_______________X
XXXXXXXXXXXXXXXXX

An optimal solution to this maze ['D', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'D', 'L', 'L', 'L', 'L']


In [13]:
show_solution(5) 

XXXXXXXXXXXXXXX
X_______x_____X
X________x_xxxX
X______x__x__xX
X___xx__x__u_xX
X___xx_x__x__xX
X__x__x____xxxX
Xe___x_x______X
X__x__________X
X______x______X
X_____x_______X
X_x___________X
XXXXXXXXXXXXXXX

An optimal solution to this maze ['L', 'L', 'D', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'U', 'L', 'L', 'L']


In [14]:
show_solution(6) 

XXXXXXXXXXXXXXXXXXXXX
X_________x____x____X
X____x_x__x_______x_X
Xu__x___x__x__x__xx_X
X____x_x__x____x_xe_X
X______x____x____x__X
XXXXXXXXXXXXXXXXXXXXX

An optimal solution to this maze ['U', 'U', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'D', 'D', 'D', 'R', 'R', 'U', 'R', 'U', 'U', 'R', 'R', 'R', 'R', 'U', 'R', 'R', 'R', 'D', 'D', 'D', 'L']


In [15]:
show_solution(7) 

XXXXXXXXXXXXXXXXXXXXXX
Xu__________x________X
X_______x___x________X
X______x____x________X
X_____x_____x________X
X____x______x________X
X___x__xxxxx__xxxxxxxX
X__x__x_____________xX
X_x_x____x____xxx___xX
X_x____x__x_________xX
X_x___________xxx___xX
X__x___x__x_________xX
X_x_x___x__x__xxx___xX
X_x__x______x_______xX
X_x____x__x___xxxx_xxX
X_____x__x____xe____xX
XXXXXXXXXXXXXXXXXXXXXX

An optimal solution to this maze ['D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'R', 'R', 'U', 'R', 'R', 'R', 'U', 'R', 'R', 'R', 'R', 'R', 'D', 'R', 'R', 'U', 'R', 'R', 'R', 'R', 'R', 'D', 'D', 'L', 'L', 'L']


In [16]:
show_solution(8) 

XXXXXXXXXXXXXXXX
Xx__x__x__x__x_X
X__x__x_x__x__xX
X_x__x___x__x__X
Xx_____x__x__x_X
X__x___xx__x__xX
X_x__x_x__x_x__X
Xx______x____x_X
X__x_xux__xx__xX
X______x_x___x_X
Xx_x_x_x__xxx__X
Xx__x_x__xxex__X
X_x__x_x__x__x_X
X_______x__x__xX
Xx_x_x_x__x__x_X
X_x___x__x__x__X
Xx_x_x_x___x__xX
XXXXXXXXXXXXXXXX

An optimal solution to this maze ['U', 'U', 'U', 'U', 'U', 'R', 'R', 'D', 'R', 'D', 'D', 'D', 'D', 'L', 'D', 'D', 'D', 'D', 'R', 'D', 'D', 'L', 'D', 'D', 'R', 'R', 'U', 'R', 'U', 'R', 'U', 'U', 'L', 'U']


In [17]:
show_solution(9)  

XXXXXXXXXXXXXXXXXXXXXX
Xu__________xx_______X
X__________x_x___e___X
X____x_x__x__x_x_____X
X__xx_x__x___x_x_____X
X__xx___x______x_____X
X___x__x_x_xx_x______X
X_x__x____xx_x_x_____X
Xx_____x__xx____x____X
X__x__x__x_x_x_x_x___X
X____x_x_____x__x_x__X
X___x___xxxxx__x_____X
X______x_______x_x__xX
X____x____________x__X
X__________________x_X
X____________________X
XXXXXXXXXXXXXXXXXXXXXX

An optimal solution to this maze ['D', 'D', 'D', 'D', 'D', 'R', 'R', 'D', 'D', 'L', 'D', 'D', 'D', 'D', 'R', 'R', 'R', 'R', 'D', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'R', 'U', 'U', 'R', 'R', 'R', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'U', 'L', 'L']


In [18]:
#Actually checking if the answers are correct for all traditional maze
#levels by testing using custom game simulation:
for level in range(1,10):
    mz = Maze(level) 
    print("Level",level,mz.check_win_from_list(Pathfinder().solve_maze(mz))) 

Level 1 True
Level 2 True
Level 3 True
Level 4 True
Level 5 True
Level 6 True
Level 7 True
Level 8 True
Level 9 True


### Mazes With Movable Blocks

These Start off about as easy as regular mazes, if not easier, but become extremely interesting in the late 40s of this game, requiring retracing your steps after altering the positions of certain blocks to make new areas accessible.

---
The first thing that occurs to me is this repetitive approach:  
1) bfs_path, treating movable blocks like empty space  
2) do moves in your bfs path until you move a movable block  
3) update the board, converting movable blocks that have become immovable to immovable blocks  
4) start over / repeat until win

There is an issue here where movable blocks may only be immovable from certain directions

This means that I may have to create some kind of object for "p" and "P" so that they can be updated whenever a block is moved.  They should also be treated differently by bfs.  It's not "immovable" or "movable".  It's more like "accessible from here", but "not accessible from there", and so this is just another thing the graph has to account for.

With this approach, looking ahead becomes necessary, because while something is accessible now, it might not be accessible later.

It feels like it's still going to be subject to issues with combinatorics...

---

It might be interesting to try reinforcement learning.  A bunch of neural networks with different rewards.

The trickiest thing about that is... what's a good reward?

In the sidescroller, Mario, it's fairly trivial.  HOW FAR TO THE RIGHT DID YOU GET?  That's a good reward.

But in these mazes with movable and immovable blocks, distance from the exit, while still something that should be a factor, isn't the whole story.

Maybe another good one is how much of the map you have explored.  It will be interesting to try this.

###

The first few maps will probably be easy because distance from the exit will be enough.

In [19]:
import keras

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [22]:
model = keras.models.Sequential()
for _ in range(2):
    model.add(keras.layers.Dense(64,activation="relu"))
model.add(keras.layers.Dense(10,activation="softmax")) #sigmoid also good / very similar

model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=['accuracy'])

#model.fit(X_train,y_train,epochs=2) 